<a href="https://colab.research.google.com/github/zoe-weinstein/CS639/blob/main/p7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
print("GPU available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

GPU available: True
GPU name: Tesla T4


In [ ]:
from huggingface_hub import login
login()

In [ ]:
!pip install bitsandbytes>=0.39.0
!pip install --upgrade accelerate transformers datasets peft trl
!pip install streamlit
!npm install -g localtunnel

  Using cached transformers-4.47.0-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
changed 22 packages in 1s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸

In [ ]:
!wget https://github.com/CS639-Data-Management-for-Data-Science/f24/raw/main/p7/transcripts.zip
!unzip transcripts.zip -d transcripts/

--2024-12-12 22:53:01--  https://github.com/CS639-Data-Management-for-Data-Science/f24/raw/main/p7/transcripts.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CS639-Data-Management-for-Data-Science/f24/main/p7/transcripts.zip [following]
--2024-12-12 22:53:01--  https://raw.githubusercontent.com/CS639-Data-Management-for-Data-Science/f24/main/p7/transcripts.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 290933 (284K) [application/zip]
Saving to: ‘transcripts.zip.9’

transcripts.zip.9   100%[===================>] 284.11K  --.-KB/s    in 0.005s  

2024-12-12 22:53:01 (54.0 MB/s) - ‘tr

## Section 1: Text Generation with a Pre-Trained LLM

### Q1.1: Load a 4-bit quantized Llama-3.2-1B-Instruct model and and its tokenizer.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


Model Used: https://huggingface.co/huihui-ai/Llama-3.2-1B-Instruct-abliterated

In [ ]:
model_id = "huihui-ai/Llama-3.2-1B-Instruct-abliterated"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    low_cpu_mem_usage=True
)
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-0

### Q1.2: Test your quantized model with different prompts (text generation).


In [ ]:
def generate_response(prompt, max_new_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
prompt = "What is there to do in San Francisco as a tourist?"
response = generate_response(prompt, max_new_tokens=250)
print(response)

What is there to do in San Francisco as a tourist??
San Francisco is a very touristy city, so there are plenty of things to do and see. Here are some ideas:
**Beaches and Boardwalks**
San Francisco has some of the most beautiful beaches and boardwalks in the world. Here are a few recommendations:
* **Haight-Ashbury**: This historic neighborhood is a must-see for any tourist. Take a stroll down the famous main street, grab a coffee at the iconic Café Gratitude, and browse the vintage shops and galleries.
* **Pier 39**: This bustling pier is a hub for tourists, but it's also a great place to grab some lunch or do some shopping. Be prepared for crowds and long lines.
* **Cable Beach**: This upscale beach is a great spot for a day of sun, fun, and relaxation. Book a cabana or rent a kayak to explore the bay.
* **North Beach**: This charming neighborhood is perfect for families and couples. Enjoy the beautiful views of the Golden Gate Bridge, grab some dinner at one of the many great restau

In [ ]:
prompt = "If I am looking to become a data scientist, what are some important skills for the industry I should have?"
response = generate_response(prompt, max_new_tokens=250)
print(response)

If I am looking to become a data scientist, what are some important skills for the industry I should have? 

Here is what you can see in the industry:
*   **Python** is a fundamental skill for data scientists. Here are some key areas to focus on: 
    - **Python programming**: Learn to write clean, readable, and efficient Python code for various tasks, including data manipulation, data analysis, machine learning, and data visualization.
    - **Data structures and algorithms**: Understand the basics of data structures (e.g., arrays, linked lists, trees, graphs) and algorithms (e.g., sorting, searching, graph traversal).
    - **Data science libraries**: Familiarize yourself with popular libraries like NumPy, pandas, and scikit-learn.
    - **Data visualization**: Learn to create informative and engaging visualizations using libraries like Matplotlib, Seaborn, and Plotly.
    - **Machine learning**: Study the basics of machine learning, including supervised and unsupervised learning, re

### Q1.3: Identify a prompt where the model fails and analyze the failure.




In [ ]:
prompt = "What is geographically interesting or unique about where the University of Wisconsin-Madison campus is located in Madison, Wisconsin?"
response = generate_response(prompt, max_new_tokens=150)
print(response)

What is geographically interesting or unique about where the University of Wisconsin-Madison campus is located in Madison, Wisconsin? The University of Wisconsin-Madison campus is located in a unique and fascinating environment that has been shaped by the city's history and culture. Here are a few examples:
* The campus is situated in a former industrial area, which has been transformed into a vibrant and sustainable urban campus. The buildings and infrastructure are designed to be environmentally friendly and low-carbon, which is a distinctive feature of the area.
* The campus is surrounded by a unique and diverse ecosystem. The buildings and surrounding landscape are surrounded by a network of green spaces, including a large public park that is a popular destination for students and local residents.
* The campus is situated in a historic and culturally rich area. The buildings and surrounding area are filled with historic and cultural landmarks, including


##### The model fails because it does not identify that UW-Madison is located in an isthmas, this is would be considered geographically interesting or unique to most people, however, because it is such a broad question the model fails to identify it. The model may have lack of relevant training data about Madison being on an isthmas or it may not realize this is a unique or interesting factor.

### Q1.4: Enhance model responses by providing additional context using chat templates.

In [ ]:
def apply_chat_template(role, prompt, max_new_tokens=100):
    messages = [{"role": "system",
                "content": role},
                {"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
role = "I am a prospective student who likes to be by the water"
role_response = apply_chat_template(role, prompt, max_new_tokens=250)
print(role_response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


system

Cutting Knowledge Date: December 2023
Today Date: 12 Dec 2024

I am a prospective student who likes to be by the wateruser

What is geographically interesting or unique about where the University of Wisconsin-Madison campus is located in Madison, Wisconsin?assistant

The University of Wisconsin-Madison campus in Madison, Wisconsin, is indeed situated in a unique and fascinating environment. Here are some geographically interesting or unique aspects of its location:

1. **Waterfront and shoreline**: As a campus situated on the shores of Lake Monona and Lake Mendonona, the University of Wisconsin-Madison has a rich aquatic history. The waterways surrounding the campus are an integral part of the landscape, with numerous boat launches, docks, and water features.

2. **Hydrology and water management**: The campus is surrounded by a complex network of waterways, including the Madison River, which provides water for irrigation, hydroelectric power, and drinking water for the universi

#### Adding that I am a prospective student the model did highlight the lakes as features of the campus, but it said they were unique because of the ecosystems that exsist. I am still looking for the model to say the campus is on an isthmas, or it is between two lakes, it is just calling it "the Madison waterfront"

## Section 2: Fine-Tuning a Pre-Trained LLM on Course Lecture Transcripts


### Q2.1: Test the model before fine-tuning.


In [ ]:
prompt = "Please explain the difference between NoSQL and SQL databases?"
print(prompt)

Please explain the difference between NoSQL and SQL databases?


In [ ]:
response = generate_response(prompt, max_new_tokens=150)
print(response)

Please explain the difference between NoSQL and SQL databases?.

Here are some examples of how to perform common operations on a table in a relational database like MySQL or PostgreSQL, and how to perform the same operations in a NoSQL database like MongoDB, Cassandra, and HSQLDB.

Here are some examples of how to perform common operations on a table in a relational database like MySQL or PostgreSQL, and how to perform the same operations in a NoSQL database like MongoDB, Cassandra, and HSQLDB.

### NoSQL Databases

**1. MongoDB**

**2. Cassandra**

**3. HSQLDB**

#### 1. MongoDB

**Example: Insert a document**

Suppose we have a simple user table with the following structure:
```sql
CREATE USER
```
```sql
CREATE


In [ ]:
role = "You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions."
role_response = apply_chat_template(role, prompt, max_new_tokens=250)
print(role_response)

system

Cutting Knowledge Date: December 2023
Today Date: 12 Dec 2024

You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.user

Please explain the difference between NoSQL and SQL databases?assistant

As an instructor of CS 639 Data Management for Data Science course at UW-Madison, I'd be happy to explain the fundamental differences between NoSQL and SQL databases.

**SQL Databases**

SQL databases, also known as Relational Databases (RDBMS), are designed to manage structured data in a fixed, tabular format. They are based on a fixed schema and use tables to store data. Here's a high-level overview of how they work:

* Each row in a table represents a single record.
* Each column represents a field or attribute of the record.
* Data is stored in a fixed, predictable order.
* Queries are based on the schema of the database.
* Data can be easily modified by changing the structure of the tables.

Example of 

### Q2.2 Fine-tune the model on course lecture transcripts with LoRA.


In [ ]:
from datasets import Dataset

from peft import LoraConfig
from transformers import TrainingArguments
from trl import SFTTrainer

In [ ]:
!ls

'=0.39.0'      transcripts.zip	    transcripts.zip.2   transcripts.zip.5   transcripts.zip.8
 sample_data   transcripts.zip.1    transcripts.zip.3   transcripts.zip.6   transcripts.zip.9
 transcripts   transcripts.zip.10   transcripts.zip.4   transcripts.zip.7


In [ ]:
!pwd

/content


In [ ]:
ls transcripts/transcripts/

'10 en-English-CS639_ SQL subqueries.txt'
'11 en-English-CS639_ SQL Joins.txt'
'12 en-English-CS639_ SQL window functions.txt'
'13 en-English-CS639_ Non-relational databases_ MongoDB.txt'
'14 en-English-CS639_ MongoDB on Docker.txt'
'15 en-English-CS639_ MongoDB API.txt'
'16 en-English-CS639_ MongoDB Operators.txt'
'17 en-English-CS639_ MongoDB Aggregation.txt'
'18 en-English-CS639_ MongoDB Geospatial Operators.txt'
'1 en-English-CS639_ Course intro.txt'
'20 en-English-CS639_ Elasticsearch intro.txt'
'21 en-English-CS639_ Elasticsearch API intro.txt'
'22 en-English-CS639_ Elasticsearch_ Boosting, highlighting, and aggregations.txt'
'23 en-English-CS639_ Elasticsearch geo queries + Kibana.txt'
'2 en-English-CS639_ Deployment (Linux Shell).txt'
'3 en-English-CS639_ Deployment (Linux Pipelines).txt'
'4 en-English-CS639_ Docker.txt'
'5 en-English-CS639_ Relational Database Management Systems (RDBMS).txt'
'6.1 en-English-CS639_ SQL 1_ Creating tables (part 1).txt'
'6.2 en-English-SQL 1_ Cre

In [ ]:
import os

In [ ]:
# Load and process the dataset
test_ratio = 0.1

# Define paths and initialize lists
transcripts_path = "/content/transcripts/transcripts"  # Adjust this to your actual folder path
train_texts = []
test_texts = []

# Get all text file names in the transcripts folder
all_files = os.listdir(transcripts_path)

file_contents = []
for file_name in all_files:
  #print(file_name)
  file_path = os.path.join(transcripts_path, file_name)
  #print(file_path)
  with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
    lines = f.readlines()
    split_idx = int(len(lines) * test_ratio)
    test_lines = lines[:split_idx]
    train_lines = lines[split_idx:]
    print(train_lines)
    print(test_lines)

    train_texts.append("".join(train_lines))
    test_texts.append("".join(test_lines))

['be a unique identifier to identify the document that is\n', 'associated with this index. Let me just use the integer 1, and the third argument is going to be the\n', 'document itself, which you need to parse as an argument to the\n', 'parameter document. So here, the document\n', 'itself is stored as a dictionary object instance\n', 'inside the variable doc, so my argument here is\n', 'simply going to be doc. Let me capture the return value into a variable called resp, and I am going to print the result field inside the\n', 'response object instance. As you can see, the\n', 'result field shows created whenever you index\n', 'a brand new document. Let me go ahead and\n', "index another document. I'm going to copy-paste\n", 'the index method. Even though the\n', 'documents look similar, they have the same author, but the text is different. This time it says it is time we learn how to handle\n', "unstructured data. I'm going to make sure\n", "to change the ID here. The ID now is going t

In [ ]:
#Convert the splits into a HuggingFace Dataset object.
train_dataset = Dataset.from_dict({"text": train_texts})
test_dataset = Dataset.from_dict({"text": test_texts})

In [ ]:
# Tokenize the train and test splits using your tokenizer.

def tokenize_data(data):
    tokenized = tokenizer(
        data["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    # Set the labels to be the same as input_ids for causal language modeling
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_train = train_dataset.map(tokenize_data, batched=True)
tokenized_test = test_dataset.map(tokenize_data, batched=True)

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

In [ ]:
tokenized_train

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 24
})

In [ ]:
tokenized_test

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 24
})

In [ ]:
# Fine-tune the model using SFTTrainer. Ensure that both training loss and validation loss are printed for each epoch.
# Use the following LoRA configurations (feel free to modify them if you'd like):
lora_config = LoraConfig(
    r=8,
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj", "o_proj", "k_proj", "v_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
)

training_args = TrainingArguments(
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    logging_dir="./logs",
    output_dir="./results",
    save_total_limit=2,
    optim="paged_adamw_8bit",
    report_to="none"
)

# Trainer object that takes care of the training process
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    args=training_args,
    peft_config=lora_config,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding

### Q2.3: Test the model after fine-tuning

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.330000,3.564228
2,2.941600,3.446091
3,2.156000,3.380636
4,1.936500,3.333345
5,2.432400,3.316694
6,1.769800,3.319584
7,2.217300,3.325745
8,2.330500,3.330155
9,2.074600,3.345561
10,2.213200,3.353799


TrainOutput(global_step=60, training_loss=2.4790423194567364, metrics={'train_runtime': 150.1677, 'train_samples_per_second': 1.598, 'train_steps_per_second': 0.4, 'total_flos': 721636546314240.0, 'train_loss': 2.4790423194567364, 'epoch': 10.0})

In [ ]:
prompt = "Please explain the difference between NoSQL and SQL databases?"
print(prompt)

In [ ]:
response = generate_response(prompt, max_new_tokens=150)
print(response)

In [ ]:
role = "You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions."
role_response = apply_chat_template(role, prompt, max_new_tokens=250)
print(role_response)

##  Section 3: Build a "Creative Writing Assistant" Using Streamlit

#### Q3: Create the "Creative Writing Assistant" application.